In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'/home/dheeraj37/Projects/Audio-Genre-Classification-1'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path



In [4]:
from cnnClassifier.constant import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_PATH,
        params_filepath = PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )    
        

        return prepare_callback_config



In [19]:
import os
import tensorflow as tf
from tensorflow import keras
import time

In [20]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [21]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2023-09-23 18:03:47,561]: INFO: common: yaml file: config/config.yaml loaded successfully
[2023-09-23 18:03:47,563]: INFO: common: yaml file: params.yaml loaded successfully
[2023-09-23 18:03:47,563]: INFO: common: created directory at: artifacts
[2023-09-23 18:03:47,563]: INFO: common: created directory at: artifacts/prepare_callbacks/checkpoint_dir
[2023-09-23 18:03:47,564]: INFO: common: created directory at: artifacts/prepare_callbacks/tensorboard_log_dir


In [9]:
import time
timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
f"tb_logs_at_{timestamp}"

'tb_logs_at_2023-09-23-18-01-09'

In [13]:
#@property
def create_ckpt_callbacks(path):
    return keras.callbacks.ModelCheckpoint(
            filepath=path,
            save_best_only=True
        )

In [14]:
path = "artifacts/prepare_callbacks/checkpoint_dir/best_model"
create_ckpt_callbacks(path)

In [ ]:
Path(path)